In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as trans
import scipy.ndimage.filters as filters

import vmdata
import more_trans
from salicae import SaliencyCAE

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline

# Load dataset

In [3]:
root = vmdata.prepare_dataset_root(9, (8, 0, 0))
dset = vmdata.VideoDataset(root)

# Gaussian Blurs

## Blurred single frame

In [102]:
def vfblur(i, std, tr):
    orig = dset[i]
    blurred = filters.gaussian_filter(dset[i], (std, std, 0.0), truncate=tr)
    plt.figure(figsize=(18, 8))
    plt.imshow(blurred)

In [112]:
interact(vfblur,
         i=widgets.BoundedIntText(value=8, min=0, max=599, step=1, description='frame ID: '),
         std=widgets.FloatSlider(value=1.5, min=0.0, max=10.0, step=0.5, continuous_update=False),
         tr=widgets.FloatSlider(value=4.0, min=0.0, max=20.0, step=1.0, continuous_update=False));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEJvdW5kZWRJbnRUZXh0KHZhbHVlPTgsIGRlc2NyaXB0aW9uPXUnZnJhbWUgSUQ6ICcsIG1heD01OTkpLCBGbG9hdFNsaWRlcih2YWx1ZT0xLjUsIGNvbnTigKY=


## Median of Gaussian

In [4]:
def vfmog(i, std, width, center_aligned):
    if center_aligned:
        window = [dset[j] for j in range(max(0, i-width//2), i+width//2)]
    else:
        window = [dset[j] for j in range(max(0, i-width), i)]
    blurred_frames = [filters.gaussian_filter(x, (std, std, 0.0), truncate=4.0)
                      for x in window]
    median_blurred = np.round(np.median(np.stack(blurred_frames), axis=0)).astype(np.uint8)
    plt.figure(figsize=(18, 8))
    plt.imshow(median_blurred)

In [5]:
interact(vfmog,
         i=widgets.BoundedIntText(value=11, min=1, max=599, step=10, description='frame ID: '),
         width=widgets.BoundedIntText(value=30, min=5, max=60, step=1, description='window: '),
         std=widgets.FloatSlider(value=1.5, min=0.0, max=10.0, step=0.5, continuous_update=False),
         center_aligned=widgets.Checkbox(value=True, description='Center aligned window'));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEJvdW5kZWRJbnRUZXh0KHZhbHVlPTExLCBkZXNjcmlwdGlvbj11J2ZyYW1lIElEOiAnLCBtYXg9NTk5LCBtaW49MSwgc3RlcD0xMCksIEZsb2F0U2xpZGXigKY=


## Mean of Gaussian

In [6]:
def vfaog(i, std, width, center_aligned):
    if center_aligned:
        window = [dset[j] for j in range(max(0, i-width//2), i+width//2)]
    else:
        window = [dset[j] for j in range(max(0, i-width), i)]
    blurred_frames = [filters.gaussian_filter(x, (std, std, 0.0), truncate=4.0)
                      for x in window]
    median_blurred = np.round(np.mean(np.stack(blurred_frames), axis=0)).astype(np.uint8)
    plt.figure(figsize=(18, 8))
    plt.imshow(median_blurred)

In [7]:
interact(vfaog,
         i=widgets.BoundedIntText(value=11, min=1, max=599, step=10, description='frame ID: '),
         width=widgets.BoundedIntText(value=30, min=5, max=60, step=1, description='window: '),
         std=widgets.FloatSlider(value=1.5, min=0.0, max=10.0, step=0.5, continuous_update=False),
         center_aligned=widgets.Checkbox(value=True, description='Center aligned window'));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEJvdW5kZWRJbnRUZXh0KHZhbHVlPTExLCBkZXNjcmlwdGlvbj11J2ZyYW1lIElEOiAnLCBtYXg9NTk5LCBtaW49MSwgc3RlcD0xMCksIEZsb2F0U2xpZGXigKY=


# Release dataset resource (DONT FORGET)

In [8]:
del dset